## Procedural Notes  

### Enzyme/Pathway Selection  
Kegg glycolysis/Glucogenesis-Reference Pathway:  

>Enzymes selected for each pathway are:  
>- Glycolysis: K00844, K11645, K01689, K16370  
>- Citric Acid Cycle: K01647, K00030, K01900, K00116  
>- Pentose Phosphate Pathway: K13937, K00036, K01807, K06859  



In [1]:
# Creating the genes, pathways and enzymes table with respective assigned columns
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
# to avoid conflict with previously built table
c.execute("""DROP TABLE genes""")
c.execute("""DROP TABLE pathways""")
c.execute("""DROP TABLE enzymes""")

# Caution: use INTEGER instead of INT for primary key generation
c.execute("""CREATE TABLE genes (id INTEGER PRIMARY KEY, name TEXT, description TEXT, organism TEXT, nucleotide_sequence TEXT
chromosome TEXT, start INT, end INT, strand VARCHAR(1), translated_sequence TEXT);""")

c.execute("""CREATE TABLE pathways (id INTEGER PRIMARY KEY, name TEXT, description TEXT);""")

c.execute("""CREATE TABLE enzymes (id INTEGER PRIMARY KEY, kegg_id TEXT, name TEXT, function TEXT, ec_number TEXT);""")

conn.commit()


### Inserting values into the pathway table
1. get the names of the pathways
2. search up the description of each pathway
3. Take in two list with equal length and insert the into the corresponding field

In [2]:
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
cmd = ''' INSERT INTO pathways (name, description) VALUES(?,?)'''

names = ["Glycolysis","Citric Acid Cycle","Pentose Phosphate"]
descriptions = ["the breakdown of glucose by enzymes, releasing energy and pyruvic acid." , "the sequence of reactions by which most living cells generate energy during the process of aerobic respiration. It takes place in the mitochondria, consuming oxygen, producing carbon dioxide and water as waste products, and converting ADP to energy-rich ATP","a metabolic pathway parallel to glycolysis, It generates NADPH and pentoses (5-carbon sugars) as well as ribose 5-phosphate, the last one a precursor for the synthesis of nucleotides"]


# Take in two list with equal length and insert the into the corresponding field
for i in range(len(names)):
    name = names[i]
    description = descriptions[i]
    temp_value = (name,description)
    c.execute(cmd,temp_value)
    conn.commit()


### Query the name and ec_number of the selected Enzymes from KEGG
1. Make a sample.list file in the current directory with the following lines  
K00844   
K11645   
K01689  
K16370  
K01647  
K00030  
K01900  
K00116  
K13937  
K00036  
K01807  
K06859  

2. In the terminal execute: `curl -g -s -S http://rest.kegg.jp/list/ko | grep -f sample.list | sed "s/cpd\://" > sample_extracted.table.txt`

3. Inspect the output file sample_extracted.table.txt  

In [3]:
# parse in the sample_extracted.table.txt for enzymes table insertion
def parse_extracted_table():
    filename = "sample_extracted.table.txt"
    in_file = open(filename, "rt")
    lines = []
    # separate each line in the for for inspection and downstream analysis
    for line in in_file:
        lines.append(line.rstrip('\n'))
        print(line)
    return lines

lines = parse_extracted_table()


# the get balues function split each line in the sample_extracted.table.txt to render the kegg_id, 
# enzyme name, function, and ec_number as separate lists
def get_values(lines):
    kegg_ids = []
    names = []
    functions = []
    ec_numbers = []
    for line in lines:
        # first, separate out the ec_number by the "[" symbol
        line_ec = line.split("[")
        ec_number = line_ec[-1]
        # separate the by the ";" symbol, the first half will be kegg_id and the name, the second part function
        line_2_part = line_ec[0].split(";")
        # line_part1 contains kegg_id and the name
        line_part1 = line_2_part[0].split()
        # line_part2 contains function description
        line_part2 = line_2_part[1].split()
#         print(line_part1)
#         print(line_part2)
        kegg_id = line_part1[0]
        # join the 1:end of part1 to form the name
        temp_name = line_part1[1:]
        temp_name[:] = [''.join(temp_name[:])]
        name = temp_name[0]
        # join all segments in part2 to form the function string
        temp_function = line_part2[:]
        temp_function[:] = [" ".join(temp_function[:])]
        function = temp_function[0]     
        kegg_ids.append(kegg_id)
        names.append(name)
        functions.append(function)
        ec_numbers.append(ec_number)
    return kegg_ids,names,functions,ec_numbers


# inspect the 3 output lists
get_values(lines)





ko:K00030	IDH3; isocitrate dehydrogenase (NAD+) [EC:1.1.1.41]

ko:K00036	G6PD, zwf; glucose-6-phosphate 1-dehydrogenase [EC:1.1.1.49 1.1.1.363]

ko:K00116	mqo; malate dehydrogenase (quinone) [EC:1.1.5.4]

ko:K00844	HK; hexokinase [EC:2.7.1.1]

ko:K01647	CS, gltA; citrate synthase [EC:2.3.3.1]

ko:K01689	ENO, eno; enolase [EC:4.2.1.11]

ko:K01807	rpiA; ribose 5-phosphate isomerase A [EC:5.3.1.6]

ko:K01900	LSC2; succinyl-CoA synthetase beta subunit [EC:6.2.1.4 6.2.1.5]

ko:K06859	pgi1; glucose-6-phosphate isomerase, archaeal [EC:5.3.1.9]

ko:K11645	fbaB; fructose-bisphosphate aldolase, class I [EC:4.1.2.13]

ko:K13937	H6PD; hexose-6-phosphate dehydrogenase [EC:1.1.1.47 3.1.1.31]

ko:K16370	pfkB; 6-phosphofructokinase 2 [EC:2.7.1.11]



(['ko:K00030',
  'ko:K00036',
  'ko:K00116',
  'ko:K00844',
  'ko:K01647',
  'ko:K01689',
  'ko:K01807',
  'ko:K01900',
  'ko:K06859',
  'ko:K11645',
  'ko:K13937',
  'ko:K16370'],
 ['IDH3',
  'G6PD,zwf',
  'mqo',
  'HK',
  'CS,gltA',
  'ENO,eno',
  'rpiA',
  'LSC2',
  'pgi1',
  'fbaB',
  'H6PD',
  'pfkB'],
 ['isocitrate dehydrogenase (NAD+)',
  'glucose-6-phosphate 1-dehydrogenase',
  'malate dehydrogenase (quinone)',
  'hexokinase',
  'citrate synthase',
  'enolase',
  'ribose 5-phosphate isomerase A',
  'succinyl-CoA synthetase beta subunit',
  'glucose-6-phosphate isomerase, archaeal',
  'fructose-bisphosphate aldolase, class I',
  'hexose-6-phosphate dehydrogenase',
  '6-phosphofructokinase 2'],
 ['EC:1.1.1.41]',
  'EC:1.1.1.49 1.1.1.363]',
  'EC:1.1.5.4]',
  'EC:2.7.1.1]',
  'EC:2.3.3.1]',
  'EC:4.2.1.11]',
  'EC:5.3.1.6]',
  'EC:6.2.1.4 6.2.1.5]',
  'EC:5.3.1.9]',
  'EC:4.1.2.13]',
  'EC:1.1.1.47 3.1.1.31]',
  'EC:2.7.1.11]'])

In [4]:
# insert values into the enzymes table with 4 equal length list names, functions, and kegg_ids and ec_numbers 
# into the corresponding columns of the enzymes table
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
cmd = ''' INSERT INTO enzymes (kegg_id, name, function, ec_number) VALUES(?,?,?,?)'''

# get the four lists
kegg_ids,names,functions,ec_numbers = get_values(lines)
# insertion
for i in range(len(names)):
    name = names[i]
    function = functions[i]
    kegg_id = kegg_ids[i]
    ec_number = ec_numbers[i]
    temp_value = (kegg_id, name, function, ec_number)
    c.execute(cmd,temp_value)
    conn.commit()

In [5]:
from Bio import Entrez
from Bio import SeqIO
#Entrez.esearch(db,term, sort)
#Entrez.efetch(db, id, rettype, retmode)
#for enzymes and genes only; Entrez doesn't have pathway info


# a helper function to search and fetch for gene, nucleotide, and protein by specifying the db_type and the term
def search_fetch(db_type,term):
    Entrez.email = 'ych323@berkeley.edu'
    handle = Entrez.esearch(db = db_type,
                       term = term,
                       sort= 'relevance',
                       idtype= 'acc')
    fetched_dict = Entrez.read(handle)
#     print(fetched_dict)
    if not fetched_dict["IdList"]:
        data = "None"
        return data
    handle = Entrez.efetch(db = db_type, id = fetched_dict["IdList"][0], rettype = 'fasta', retmode = 'text')
    data = handle.read()
    return data


# build a term list of all the search "terms species[ORGN] enzyme_name"
# build the 3 * 12 entries of name, organism, and description of the gene
species_list = ["Drosophila","Escherichia coli","Homo sapiens"]
ter_list = [str(species) + "[ORGN]" + " " for species in species_list ]
term_list = [str(term) + str(name) for term in ter_list for name in names]
name_list = [str(name) for term in ter_list for name in names]
organism_list = [str(species) for species in species_list for name in names]
descriptions_list = [str(function) for species in species_list for function in functions]

# inspect the built lists 
print(term_list)
print(name_list)
print(organism_list)
print(descriptions_list)


['Drosophila[ORGN] IDH3', 'Drosophila[ORGN] G6PD,zwf', 'Drosophila[ORGN] mqo', 'Drosophila[ORGN] HK', 'Drosophila[ORGN] CS,gltA', 'Drosophila[ORGN] ENO,eno', 'Drosophila[ORGN] rpiA', 'Drosophila[ORGN] LSC2', 'Drosophila[ORGN] pgi1', 'Drosophila[ORGN] fbaB', 'Drosophila[ORGN] H6PD', 'Drosophila[ORGN] pfkB', 'Escherichia coli[ORGN] IDH3', 'Escherichia coli[ORGN] G6PD,zwf', 'Escherichia coli[ORGN] mqo', 'Escherichia coli[ORGN] HK', 'Escherichia coli[ORGN] CS,gltA', 'Escherichia coli[ORGN] ENO,eno', 'Escherichia coli[ORGN] rpiA', 'Escherichia coli[ORGN] LSC2', 'Escherichia coli[ORGN] pgi1', 'Escherichia coli[ORGN] fbaB', 'Escherichia coli[ORGN] H6PD', 'Escherichia coli[ORGN] pfkB', 'Homo sapiens[ORGN] IDH3', 'Homo sapiens[ORGN] G6PD,zwf', 'Homo sapiens[ORGN] mqo', 'Homo sapiens[ORGN] HK', 'Homo sapiens[ORGN] CS,gltA', 'Homo sapiens[ORGN] ENO,eno', 'Homo sapiens[ORGN] rpiA', 'Homo sapiens[ORGN] LSC2', 'Homo sapiens[ORGN] pgi1', 'Homo sapiens[ORGN] fbaB', 'Homo sapiens[ORGN] H6PD', 'Homo sap

In [6]:
# Get the nucleotide sequences of each enzyme through searching entrez and fetch the most relevent result
def get_nucleotide_sequence(term_list):
    nucleotide_list = []
    for term in term_list:
        print(term)
        data = search_fetch("nucleotide",term)
        # If data is "None", then the entrez search has fetched no id and the idlist is empty, and 
        # so cannot be parsed by SeqIO, so should be directly append to the nucleotide list
        if data == "None":
            nucleotide_list.append(data)
            
        # If data is not "None", then the search_and_fetch function return the most relevent sequence, wrtie data into a
        # fasta file and then use SeqIO.parse to get the sequence to a string for sqlite insertion
        f = open('temp.fa','w')
        f.write(data)
        f.close()
        print("parsed")
        for seq in SeqIO.parse("temp.fa", "fasta"):
            nucleotide_list.append(str(seq.seq))
    return nucleotide_list

nucleotide_list = get_nucleotide_sequence(term_list)
# inspect first three entries of the nucleotide_list
print(nucleotide_list[1:4])


Drosophila[ORGN] IDH3
parsed
Drosophila[ORGN] G6PD,zwf
parsed
Drosophila[ORGN] mqo
parsed
Drosophila[ORGN] HK
parsed
Drosophila[ORGN] CS,gltA
parsed
Drosophila[ORGN] ENO,eno
parsed
Drosophila[ORGN] rpiA
parsed
Drosophila[ORGN] LSC2
parsed
Drosophila[ORGN] pgi1
parsed
Drosophila[ORGN] fbaB
parsed
Drosophila[ORGN] H6PD
parsed
Drosophila[ORGN] pfkB
parsed
Escherichia coli[ORGN] IDH3
parsed
Escherichia coli[ORGN] G6PD,zwf
parsed
Escherichia coli[ORGN] mqo
parsed
Escherichia coli[ORGN] HK
parsed
Escherichia coli[ORGN] CS,gltA
parsed
Escherichia coli[ORGN] ENO,eno
parsed
Escherichia coli[ORGN] rpiA
parsed
Escherichia coli[ORGN] LSC2
parsed
Escherichia coli[ORGN] pgi1
parsed
Escherichia coli[ORGN] fbaB
parsed
Escherichia coli[ORGN] H6PD
parsed
Escherichia coli[ORGN] pfkB
parsed
Homo sapiens[ORGN] IDH3
parsed
Homo sapiens[ORGN] G6PD,zwf
parsed
Homo sapiens[ORGN] mqo
parsed
Homo sapiens[ORGN] HK
parsed
Homo sapiens[ORGN] CS,gltA
parsed
Homo sapiens[ORGN] ENO,eno
parsed
Homo sapiens[ORGN] rpiA
p

In [11]:
from Bio import Entrez
from Bio import SeqIO

# following similar approach as get_nucleotide to get the protein sequence from entrez search and fetch
def get_protein(term_list):
    protein_list = []
    for term in term_list:
        print(term)
        data = search_fetch("protein",term)
        f = open('temp.fa','w')
        f.write(data)
        f.close()
        print("parsed")
        for seq in SeqIO.parse("temp.fa", "fasta"):
            protein_list.append(str(seq.seq))
    return protein_list
protein_list = get_protein(term_list)

#inspect the output protein list
print(protein_list[1:3])


Drosophila[ORGN] IDH3
parsed
Drosophila[ORGN] G6PD,zwf
parsed
Drosophila[ORGN] mqo
parsed
Drosophila[ORGN] HK
parsed
Drosophila[ORGN] CS,gltA
parsed
Drosophila[ORGN] ENO,eno
parsed
Drosophila[ORGN] rpiA
parsed
Drosophila[ORGN] LSC2
parsed
Drosophila[ORGN] pgi1
parsed
Drosophila[ORGN] fbaB
parsed
Drosophila[ORGN] H6PD
parsed
Drosophila[ORGN] pfkB
parsed
Escherichia coli[ORGN] IDH3
parsed
Escherichia coli[ORGN] G6PD,zwf
parsed
Escherichia coli[ORGN] mqo
parsed
Escherichia coli[ORGN] HK
parsed
Escherichia coli[ORGN] CS,gltA
parsed
Escherichia coli[ORGN] ENO,eno
parsed
Escherichia coli[ORGN] rpiA
parsed
Escherichia coli[ORGN] LSC2
parsed
Escherichia coli[ORGN] pgi1
parsed
Escherichia coli[ORGN] fbaB
parsed
Escherichia coli[ORGN] H6PD
parsed
Escherichia coli[ORGN] pfkB
parsed
Homo sapiens[ORGN] IDH3
parsed
Homo sapiens[ORGN] G6PD,zwf
parsed
Homo sapiens[ORGN] mqo
parsed
Homo sapiens[ORGN] HK
parsed
Homo sapiens[ORGN] CS,gltA
parsed
Homo sapiens[ORGN] ENO,eno
parsed
Homo sapiens[ORGN] rpiA
p

In [12]:
# to get the chromosome, start, and end position and possibly other information from the gene entrez search
# CAUTION: hasn't implemented through iteration yet
from Bio import Entrez
# function to separate lines of the gene search and fetch output txt file
def parse_gene_entrez():
    filename = "temp_gene_entrez.txt"
    in_file = open(filename, "rt")
    lines = []
    # separate each line in the for for inspection and downstream analysis
    for line in in_file:
        lines.append(line.rstrip('\n'))
    return lines



def get_location(term_list):   
    loci = []
    annotations = []
    Entrez.email = 'ych323@berkeley.edu'
    for term in term_list:
        handle = Entrez.esearch(db = 'gene',
                           term = 'homo sapiens[ORGN] G6PD',
                           sort= 'relevance',
                           idtype= 'acc')

        fetched_dict = Entrez.read(handle)
        handle = Entrez.efetch(db = 'gene', id = fetched_dict["IdList"][0], rettype = 'fasta', retmode = 'text')
        data = handle.read()
        f = open('temp_gene_entrez.txt','w')
        f.write(data)
        f.close()
        lines = parse_gene_entrez()
        # Location and annotation are in the 5 and 6 of the lines list 
        locus = lines[5]
        annotation = lines[6]
        loci.append(locus)
        annotations.append(annotation)
    return loci, annotations

loci, annotations = get_location(term_list)

print(loci)
print(annotations)


['Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromosome: X; Location: Xq28', 'Chromoso